KWIKVAL - Rui Cruzeiro - Dezembro 2022

O objectivo deste documento é a criação de uma ferramenta simples de avaliação imobiliária (Kwikval) com recurso a Machine Learning. Os passos dados pela ferramenta são os seguintes:
1. INPUTS: Pedir ao utilizador um distrito, um concelho, coordenadas geográficas, um tipo de imóvel e a sua Área Bruta Dependente (conforme registado na Caderneta Predial);
2. API: Pesquisar os anúncios do Idealista que mais se assemelham ao fornecido pelo utilizador. Para tal, foi criada uma conta no RapidAPI que permite 500 pesquisas gratuitas por mês na API do Idealista;
3. HOMOGENEIZAÇÃO: Fazer homogeneização de áreas e dedução da margem de negociação;
4. MACHINE LEARNING: Aplicação de Regressão Linear com o Scikit-Learn (o que corresponde à aplicação do Método Comparativo de Mercado (método tradicional de avaliação imobiliária).

In [ ]:
import requests
from concelhos import dados_concelhos

## INPUTS

O utilizador escolhe um distrito e um concelho. Estes serão usados para construir o url do Casa Sapo a usar no request. Para isso, foi construído dicionário com os concelhos para cada distrito, com informação obtida na Internet.

In [1]:
# Exemplo de trabalho

user_distrito = 'Viseu'
user_concelho = 'São Pedro do Sul'

user_lat = 40.759687500941105
user_long = -8.06400884491926

# Latitudes e longitudes extremas de Portugal
max_lat_N = 42.153977917467444
max_lat_S = 32.63296026655531
max_long_W = -31.274994594732977
max_long_E = -6.189362584898822

As coordenadas fornecidas pelo utilizador não podem ultrapassar os limites geográficos de Portugal.

In [ ]:
# Pontos geográficos extremos de Portugal
max_lat_N = 42.153977917467444
max_lat_S = 32.63296026655531
max_long_W = -31.274994594732977
max_long_E = -6.189362584898822

## API

Para obter uma lista de imóveis à venda numa zona, é necessário fazer um GET request à API introduzindo o ID e o nome da zona. O ID da zona corresponde à concatenação apresentada no campo seguinte, em que o dicionário `dados_concelhos`, construído manualmente, faz corresponder um número ao concelho inserido pelo utilizador. Na maior parte dos concelhos, esse número corresponde ao código da Autoridade Tributária para o concelho. O nome da zona é obtido na própria API partindo do ID.

Obtenção do ID da zona:

In [62]:
zona_id = '0-EU-PT-' + dados_concelhos[user_distrito][user_concelho]
zona_id

'0-EU-PT-18-16'

Obtenção do nome da zona:

In [63]:
# Obtém json da API com informação sobre IDs e nomes de zonas similares ao do concelho fornecido pelo utilizador

url = "https://idealista2.p.rapidapi.com/auto-complete"
querystring = {"prefix":user_concelho, "country":"pt"}

headers = {
    "X-RapidAPI-Key": "d72b63a893mshc0418976a6d125dp18a26ejsn1323ffe12360",
    "X-RapidAPI-Host": "idealista2.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)
zonas = response.json()

# Compara as zonas obtidas acima com a zona pretendida através do ID da zona e obtém o nome

for zona in zonas['locations']:
    try:
        if zona['locationId'] == zona_id:
            zona_nome = zona['name']
    except Exception:
        pass
    
zona_nome

'São Pedro do Sul, Viseu'

Com o ID e o nome, é possível obter a lista das propriedades no concelho escolhido. Esta informação é actualizada diariamente pelo Idealista.

In [61]:
url = "https://idealista2.p.rapidapi.com/properties/list"

querystring = {
    "locationId":zona_id,
    "locationName":zona_nome,
    "operation":"sale",
    "numPage":"1",
    "maxItems":"40",
    "sort":"asc",
    "locale":"pt",
    "country":"pt"
}

headers = {
    "X-RapidAPI-Key": "d72b63a893mshc0418976a6d125dp18a26ejsn1323ffe12360",
    "X-RapidAPI-Host": "idealista2.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

response.json()

{'elementList': [{'propertyCode': '32302498',
   'thumbnail': 'https://img3.idealista.pt/blur/WEB_LISTING-M/0/id.pro.pt.image.master/26/c7/bf/199045148.jpg',
   'numPhotos': 7,
   'price': 5999.0,
   'propertyType': 'countryHouse',
   'operation': 'sale',
   'size': 60.0,
   'exterior': False,
   'rooms': 2,
   'bathrooms': 1,
   'address': 'Lugar Aldeia de Sete Fontes s/n',
   'province': 'Viseu',
   'municipality': 'São Martinho das Moitas e Covas do Rio',
   'country': 'pt',
   'locationId': '0-EU-PT-18-16-001-22',
   'latitude': 40.9025191,
   'longitude': -8.0566958,
   'showAddress': True,
   'url': 'https://www.idealista.pt/imovel/32302498/',
   'description': 'Prédio devoluto, numa zona paradísiaca: aldeia de Sete Fontes, S. Pedro do Sul. Rés do chão e 1º andar, para reconstruir; paredes em pedra, vista magnífica! Junto à fonte de água fresca e cristalina, potável, vinda das entranhas da Serra de S. Macário! Próximo, com bons acessos, de S. Pedro do Sul, Castro Daire e Viseu; a